In [6]:
import json
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Modelo BioBERTpt
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[0, 0, :].numpy()

# Tradução das categorias do catalão
categorias_traduzidas = {
    'PRINCIPIS ACTIUS': 'Princípios activos',
    'ETIOPATOGÈNIA': 'Etiopatogénese',
    'TRACTAMENT': 'Tratamento',
    'CLÍNICA': 'Clínica',
    'EPIDEMIOLOGIA': 'Epidemiologia',
    'PREVENCIÓ': 'Prevenção',
    'ENTORN SOCIAL': 'Contexto social',
    'CONCEPTES GENERALS': 'Conceitos gerais',
    'DIAGNÒSTIC': 'Diagnóstico',
}

EXEMPLOS_CATEGORIAS = {
    "Governança e administração pública": [
        "Accountability", "Acesso à informação", "Agenda estratégica", "Agenda política",
        "Alocação de recursos", "Colegiado", "Compliance", "Controle", "Controle social",
        "Pacto", "Pactuação", "Política"
    ],
    "Políticas e intervenções na saúde pública": [
        "Ações de saúde", "Análise de situação em saúde", "Avaliação em saúde",
        "Planejamento em saúde", "Política pública de saúde", "Sala de situação de saúde",
        "Serviços de saúde", "Valor público em saúde"
    ],
    "Monitoramento e avaliação": [
        "Acompanhamento", "Análise de indicadores", "Auditoria", "Avaliação de impacto",
        "Avaliação de processo", "Avaliação de resultados", "Indicador", "Linha de base"
    ],
    "Gestão de dados e informação": [
        "Agregação de dados", "Análise de dados", "Banco de dados", "Base de dados",
        "Big data", "Ciência da informação", "Coleta de dados", "Confiabilidade",
        "Metadados", "Qualidade da informação", "Segurança da informação"
    ],
    "Planeamento e estratégia": [
        "Análise de viabilidade", "Análise estratégica", "Análise ex-ante", "Cadeia de valor",
        "Ciclo de políticas públicas", "Objetivo", "Cenário", "Orçamento", "Plano",
        "Referencial estratégico"
    ],
    "Metodologias e ferramentas": [
        "Benchmarking", "Brainstorming", "Cálculo do indicador", "Machine learning",
        "Mensuração", "Método", "Metodologia", "Parâmetro", "Unidade de medida"
    ],

    "Componentes e conceitos farmacológicos": [
        "Analgésico", "Antibiótico", "Antiviral", "Antisséptico", "Anti-inflamatório",
        "Anticoagulante", "Antihistamínico", "Corticosteróide", "Diurético", "Sedativo"
    ],

    "Princípios activos": [
        "Acalabrutinib", "Anakinra", "Azitromicina", "Baricitinib", "Bevacizumab",
        "Dexametasona", "Hidroxicloroquina", "Remdesivir", "Ritonavir", "Tocilizumab"
    ],

    "Etiopatogénese": [
        "Apicoplasto", "Apoptose", "Bioadesão", "Bainha de mielina",
        "Perlecan", "DNA", "ARN", "ARN missatger",
        "(herpes) zóster", "Araraquara", "MERS-CoV", "Avium",
        "Vírus Cantagalo", "Vírus amarílico", "Vírus respiratório sincicial",
        "Rato de rabo peludo", "Genes do imprinting"
    ],

    "Tratamento": [
        "Artrodistrator", "Bipsoro eletrônico", "Bisturi óptico",
        "Pinça óptica", "Trocáter", "Videodermatoscópio",
        "Radiocirurgia", "Pleurodese", "Terapia gênica", "Cirurgia a céu aberto", "Enxerto orgânico"
    ],

    "Clínica": [
        "Adenoma", "Acne", "Abcesso", "Acidose metilmalônica",
        "Alergia asmática", "Angina instável",
        "Atrofia espinhal progressiva tardia", "Barriga de cerveja",
        "Câncer de boca", "Bulimia nervosa", "Câncer de mama",
        "Poliparasitismo", "Transtorno do pânico",
        "Síndrome hemolítica urêmica", "Tumor de Walke", "Esgotamento mental"
    ],

    "Conceitos gerais": [
        "(d)escamação", "Gram-negativo", "Gram-positivo",
        "Epilepsia menor", "Abdominal", "Aberrante", "Absorção",
        "Adolescente", "Adrenérgico", "Aerossol", "Afinidade",
        "Agente", "Adstringente", "Acidez", "Adaptação",
        "Adenite", "Aditivo", "Adjuvante", "Adsorção", "Acumulação",
        "Anumérica", "Acomodação", "Acidental", "Acatisia", "Abstinência"
    ],

    "Diagnóstico": [
        "Análise biomecânica", "Baciloscopia", "Bio feedback", "Ceratoscopia",
        "Pleurodese", "Radiocirurgia", "Terapia gênica", "Videomicroscopia",
        "PCR", "Prova PCR"
    ],

    "Epidemiologia": [
        "MERS‑CoV", "Vírus respiratório sincicial", "(herpes) zóster",
        "Poliparasitismo", "Vírus Cantagalo", "Vírus amarílico"
    ],

    "Prevenção": [
        "EPI", "Máscara FFP", "Mascareta quirúrgica", "Pauta vacunal"
    ],

    "Anatomia": [
        "Sistema nervoso parassimpático", "Sistema nervoso periférico", "Terminal nervoso simpático", "Córtex cingulado",
        "Ilhota pancreática", "Cavidade pleural", "Líquido cefalorraquidiano", "Sistema endócrino", "Membrana celular",
        "Núcleo paraventricular", "Núcleo mediano da rafe", "Sistema límbico", "Veia porta"
    ]


}


# Calcular embeddings médios para cada categoria
categoria_embeddings = {
    cat: np.stack([get_embedding(e) for e in exemplos]).mean(axis=0)
    for cat, exemplos in EXEMPLOS_CATEGORIAS.items()
}

with open("dicionario_final_similares.json", encoding="utf-8") as f:
    termos = json.load(f)

resultado = defaultdict(dict)

for termo, dados in termos.items():
    if "Categoria" in dados:
        cat_catalan = dados["Categoria"]
        cat_pt = categorias_traduzidas.get(cat_catalan, cat_catalan)
    else:
        termo_emb = get_embedding(termo)
        sims = {cat: cosine_similarity([termo_emb], [emb])[0, 0]
                for cat, emb in categoria_embeddings.items()}
        cat_pt = max(sims, key=sims.get)
    resultado[cat_pt][termo] = {k: v for k, v in dados.items() if k != "Categoria"}

with open("dicionario_final_categorizado.json", "w", encoding="utf-8") as f:
    json.dump(resultado, f, ensure_ascii=False, indent=2)

print("Classificação concluída! Resultado dicionario_final_categorizado.json")


c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Classificação concluída! Resultado dicionario_final_categorizado.json


In [7]:
with open("dicionario_final_categorizado.json", encoding="utf-8") as f:
    dados = json.load(f)

for categoria, termos in dados.items():
    print(f"Categoria: {categoria}")
    print("Termos associados:", list(termos.keys()))
    print()

Categoria: Conceitos gerais
Termos associados: ['ACA', 'Alta', 'ARN', 'ARNm', 'ARV', 'Ensaio clínico aleatorizado', 'Ensaio clínico em ocultaçao', 'Ensaio clínico em dupla ocultação', 'Ensaio clínico em tripla ocultação', 'Ensaio clínico controlado', 'Ensaio de intervenção comunitária', 'Careta', 'Càrrega vírica', 'Cas possible', 'Cèl·lula B', 'Cèl·lula T', 'Citocina', 'Contagi mediat', 'Convidecia', 'Covid', 'Covid-19', 'Covid persistent', 'Defesas', 'Desescalament', 'Distanàsia', 'Distància física', 'Distanciament físic', 'Èmesi', 'ERTO', 'Acontecimento adverso', 'Estudo clínico', 'Estudo de fase 0', 'Estudo farmacológico', 'Estudo exploratório', 'Estudo de confirmação', 'Estudo de vigilância pós-comercialização', 'Estudo pré-clínico', 'GCE', 'Goteta', 'Grup de convivència', 'Grup de convivència ampliada', 'Grup de convivència estable', 'IA 1', 'IA 2', 'IECA', 'Imune', 'Imunitário', 'Imunidade', 'Imunidade ativa', 'Imunidade adquirida', 'Immunitat adquirida', 'Immunitat col·lectiva',

In [8]:
#versão para inserir a categoria nos dicionários dos termos mesmo como entrada

with open("dicionario_final_similares.json", encoding="utf-8") as f:
    termos = json.load(f)

resultado = {}

for termo, dados in termos.items():
    # Se já tem categoria, traduz 
    if "Categoria" in dados:
        cat_catalan = dados["Categoria"]
        cat_pt = categorias_traduzidas.get(cat_catalan, cat_catalan)
        dados["Categoria"] = cat_pt
    else:
        termo_emb = get_embedding(termo)
        sims = {cat: cosine_similarity([termo_emb], [emb])[0, 0]
                for cat, emb in categoria_embeddings.items()}
        cat_pt = max(sims, key=sims.get)
        dados["Categoria"] = cat_pt
    resultado[termo] = dados

with open("dicionario_categorizado_v2.json", "w", encoding="utf-8") as f:
    json.dump(resultado, f, ensure_ascii=False, indent=2)

print("Classificação concluída! Resultado dicionario_categorizado_v2.json")

Classificação concluída! Resultado dicionario_categorizado_v2.json
